<a href="https://colab.research.google.com/github/fmathews11/College_Basketball_Functions/blob/main/Pulling_Gameplan_From_Kenpom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kenpompy
!pip install mechanicalsoup
import mechanicalsoup,pandas as pd,numpy as np, warnings,seaborn as sns, matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from kenpompy.utils import login
warnings.filterwarnings("ignore")

In [5]:

def get_team_gameplan(browser,team):

  #Pull in HTML
  url = 'https://kenpom.com/gameplan.php?team='+str(team)
  browser.open(url)
  raw_source = browser.get_current_page()
  table = raw_source.find_all('table')
  df = pd.read_html(str(table[0]))[0]

  #Let's Clean it up
  df.columns = [i[0] +  "_" +i[1] for i in df.columns.tolist()]
  df = df.drop([df.columns[i] for i in [1,7,12,14,18]],axis = 1)

  #Column name cleaning
  for i in df.columns:
    if "." in i:
      df = df.rename(columns = {i:i[:-2]})
  for i in df.columns:
    if "Unnamed" in i:
      df = df.rename(columns = {i:i.split()[-1]})
  for i in df.columns:
    if "level" in i:
      df = df.rename(columns = {i:i.split("_")[-1]})


  #Remove Extra Junk From the end
  cutoff_value = np.where(df.Date == "Correlations (R x 100)")[0][0]
  df = df.iloc[:cutoff_value,:]

  #Splitting the results column
  result_vals = [i for i in df.Result.tolist()]
  df.Result = [str(i[0]) for i in result_vals]
  df['Score'] = [str(i.split(',')[-1]) for i in result_vals]

  col_names = ['Date',
              'Opponent',
              'Result',
              'Location',
              'Pace',
              'Offense_Eff',
              'Offense_eFG%',
              'Offense_TO%',
              'Offense_OR%',
              'Offense_FTR',
              'Offense_2P%',
              'Offense_3P%',
              'Offense_3PA%',
              'Defense_Eff',
              'Defense_eFG%',
              'Defense_TO%',
              'Defense_OR%',
              'Defense_FTR',
              'Defense_2P',
              'Defense_2P%',
              'Defense_3P',
              'Defense_3P%',
              'Defense_3PA%',
              'Score']
  df.columns = col_names
  df.Result = df.Result.map({"W":1,"L":0})

  #Coerce all strings to numeric type for computation
  for i in df.columns:
   try:
     df[i] = pd.to_numeric(df[i])
   except:
     continue

  return df


In [ ]:
df = get_team_gameplan(browser,'Purdue')
df

IndexError: ignored

In [7]:
import kenpompy.summary as kp
sum_df = kp.get_efficiency(browser,2022)
sum_df_ = sum_df.copy()
sum_df[sum_df.Team == 'Purdue']

,Team,Conference,Tempo-Adj,Tempo-Adj.Rank,Tempo-Raw,Tempo-Raw.Rank,Avg. Poss Length-Offense,Avg. Poss Length-Offense.Rank,Avg. Poss Length-Defense,Avg. Poss Length-Defense.Rank,Off. Efficiency-Adj,Off. Efficiency-Adj.Rank,Off. Efficiency-Raw,Off. Efficiency-Raw.Rank,Def. Efficiency-Adj,Def. Efficiency-Adj.Rank,Def. Efficiency-Raw,Def. Efficiency-Raw.Rank
254,Purdue,B10,66.0,243,66.7,259,17.8,205,18.2,314,121.8,3,119.2,2,99.1,101,101.8,184


In [12]:
team_1 = 'Purdue'
team_2 = 'Michigan St.'

sum_df = sum_df_.copy()
for i in sum_df.columns:
  try:
    sum_df[i] = pd.to_numeric(sum_df[i])
  except:
    pass
sum_df.columns = ['_'.join(i.split()) for i in sum_df.columns.tolist()]
sum_df.columns = [i.replace(".","") for i in sum_df.columns]
sum_df.columns = [i.replace("-","_") for i in sum_df.columns]
off_eff_avg = round(sum_df.Off_Efficiency_Adj.mean(),2)
def_eff_avg = round(sum_df.Def_Efficiency_Adj.mean(),2)

team_1_off_eff = sum_df.loc[sum_df.Team == team_1].Off_Efficiency_Adj.item()
team_1_def_eff = sum_df.loc[sum_df.Team == team_1].Def_Efficiency_Adj.item()
team_2_off_eff = sum_df.loc[sum_df.Team == team_2].Off_Efficiency_Adj.item()
team_2_def_eff = sum_df.loc[sum_df.Team == team_2].Def_Efficiency_Adj.item()
team_1_tempo = sum_df.loc[sum_df.Team == team_1].Tempo_Adj.item()
team_2_tempo = sum_df.loc[sum_df.Team == team_2].Tempo_Adj.item()
adj_tempo = round(np.mean([team_1_tempo,team_2_tempo]))

team_1_off_pct = (team_1_off_eff/off_eff_avg)-1
team_1_off_pct_adjustment_factor = 1+round(team_1_off_pct + ((team_2_def_eff/def_eff_avg) - 1),2)
team_1_off_adj = round(off_eff_avg * team_1_off_pct_adjustment_factor,2)
print(f"{team_1}'s expected efficiency is {team_1_off_adj}")

team_2_off_pct = (team_2_off_eff/off_eff_avg)-1
team_2_off_pct_adjustment_factor = 1+round(team_2_off_pct + ((team_1_def_eff/def_eff_avg) - 1),2)
team_2_off_adj = round(off_eff_avg * team_2_off_pct_adjustment_factor,2)
print(f"{team_2}'s expected efficiency is {team_2_off_adj}")
#print(f"{team_1} averages {round(team_1_tempo)} possessions per game")
#print(f"{team_2} averages {round(team_2_tempo)} possessions per game")
print(f"{team_1} is expected to score {round((adj_tempo)*(team_1_off_adj/100))} points")
print(f"{team_2} is expected to score {round((adj_tempo)*(team_2_off_adj/100))} points")

Purdue's expected efficiency is 115.42
Michigan St.'s expected efficiency is 107.17
Purdue is expected to score 77 points
Michigan St. is expected to score 72 points
